In [1]:
import pygame
import neat
import time
import os
import random
import pickle
pygame.font.init()

pygame 2.1.2 (SDL 2.0.18, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
SCREEN_WIDTH = 500
SCREEN_HEIGHT = 800
GEN = 0

In [3]:
BIRD_IMGS=[pygame.transform.scale2x(pygame.image.load(os.path.join("imgs","bird1.png"))), pygame.transform.scale2x(pygame.image.load(os.path.join("imgs","bird2.png"))), pygame.transform.scale2x(pygame.image.load(os.path.join("imgs","bird3.png")))]
PIPE_IMG = pygame.transform.scale2x(pygame.image.load(os.path.join("imgs","pipe.png")))
FLOOR_IMG = pygame.transform.scale2x(pygame.image.load(os.path.join("imgs","base.png")))
BG_IMG = pygame.transform.scale2x(pygame.image.load(os.path.join("imgs","bg.png")))

STAT_FONT = pygame.font.SysFont("comicsansms", 20)

In [4]:
#bird options
bird_acceleration = 3
bird_max_displacement = 16

In [5]:
#Bird objects moving
class Bird:
    IMGS= BIRD_IMGS
    MAX_ROTATION = 25
    ROT_VEL = 20
    ANIMATION_TIME = 5
    def __init__(self, x_position, y_position):
        self.x = x_position #starting x position
        self.y = y_position #starting y position
        self.tilt = 0 #starting flying angle 
        self.time = 0 #
        self.vel = 0
        self.height = self.y
        self.img_count = 0
        self.img = self.IMGS[0]
        self.acceleration = 3
        
    def jump(self):
        self.vel = -10.5
        self.time = 0
        self.height = self.y
        
    def move(self):
        self.time += 1
        
        d = self.vel*self.time + (1/2)* bird_acceleration * self.time**2
        
        if d >= bird_max_displacement:
            d = bird_max_displacement
        if d<0:
            d -= 2
            
        self.y = self.y + d
        
        if d<0 or self.y < self.height + 50:
            if self.tilt < self.MAX_ROTATION:
                self.tilt = self.MAX_ROTATION
        else:
            if self.tilt > -90:
                self.tilt -= self.ROT_VEL
                
    def draw(self, screen):
        self.img_count += 1
        
        if self.img_count < self.ANIMATION_TIME:
            self.img = self.IMGS[0]
        elif self.img_count < self.ANIMATION_TIME*2:
            self.img = self.IMGS[1]
        elif self.img_count < self.ANIMATION_TIME*3:
            self.img = self.IMGS[2]
        elif self.img_count < self.ANIMATION_TIME*4:
            self.img = self.IMGS[1]
        elif self.img_count == self.ANIMATION_TIME*4+1:
            self.img = self.IMGS[0]
            self.img_count = 0
            
        if self.tilt <= -80:
            self.img = self.IMGS[1]
            self.img_count = self.ANIMATION_TIME*2
            
        rotated_image = pygame.transform.rotate(self.img, self.tilt)
        new_rect = rotated_image.get_rect(center=self.img.get_rect(topleft = (self.x, self.y)).center)
        screen.blit(rotated_image, new_rect.topleft)
        
    def get_mask(self):
        return pygame.mask.from_surface(self.img)


In [6]:
class Pipe:
    GAP = 200
    VEL = 5
    
    def __init__(self,x):
        self.x = x
        self.height = 0
        self.gap = 100
        
        self.top = 0
        self.bottom = 0
        self.PIPE_TOP = pygame.transform.flip(PIPE_IMG, False, True)
        self.PIPE_BOTTOM = PIPE_IMG
        
        self.passed = False
        self.set_height()
        
    def set_height(self):
        self.height = random.randrange(50, 450)
        self.top = self.height - self.PIPE_TOP.get_height()
        self.bottom = self.height + self.GAP
        
    def move(self):
        self.x -= self.VEL
        
    def draw(self, screen):
        screen.blit(self.PIPE_TOP, (self.x, self.top))
        screen.blit(self.PIPE_BOTTOM, (self.x, self.bottom))
        
    def collide(self, bird):
        bird_mask = bird.get_mask()
        top_mask = pygame.mask.from_surface(self.PIPE_TOP)
        bottom_mask = pygame.mask.from_surface(self.PIPE_BOTTOM)
        
        top_offset = (self.x - bird.x, self.top - round(bird.y))
        bottom_offset = (self.x - bird.x, self.bottom - round(bird.y))
        
        b_point = bird_mask.overlap(bottom_mask, bottom_offset)
        t_point = bird_mask.overlap(top_mask, top_offset)
        
        if t_point or b_point:
            return True
        
        return False
    

In [7]:
class Base:
    VEL = 5
    WIDTH = FLOOR_IMG.get_width()
    IMG = FLOOR_IMG
    
    def __init__(self, y):
        self.y = y
        self.x1 = 0
        self.x2 = self.WIDTH
        
    def move(self):
        self.x1 -= self.VEL
        self.x2 -= self.VEL
        
        if self.x1 + self.WIDTH < 0:
            self.x1 = self.x2 + self.WIDTH
            
        if self.x2 + self.WIDTH < 0:
            self.x2 = self.x1 + self.WIDTH
            
    def draw(self, screen):
        screen.blit(self.IMG, (self.x1, self.y))
        screen.blit(self.IMG, (self.x2, self.y))

In [8]:
#draw the screen for the game
def draw_screen(screen, birds, pipes, base, score, GEN):
    screen.blit(BG_IMG, (0,0))
    for pipe in pipes:
        pipe.draw(screen)
     
    text = STAT_FONT.render("Score: "+str(score), 1, (255,255,255))
    screen.blit(text, (SCREEN_WIDTH - 10 - text.get_width(), 10))
    
    text = STAT_FONT.render("GEN: "+str(GEN), 1, (255,255,255))
    screen.blit(text, (10, 10))
    
    base.draw(screen)
        
    for bird in birds:
        bird.draw(screen)
        
    pygame.display.update()

#Runs main loop of game
def main(genomes, config):
    global GEN
    GEN +=1
    nets = []
    ge = []
    birds = []
    
    for _, g in genomes:
        net = neat.nn.FeedForwardNetwork.create(g, config)
        nets.append(net)
        birds.append(Bird(230, 350))
        g.fitness = 0
        ge.append(g)
        
    
    base = Base(730)
    pipes = [Pipe(600)]
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    clock = pygame.time.Clock()
    
    score = 0
    
    run = True
    while run:
        clock.tick(80)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                pygame.quit()
                quit()
                
                
        pipe_ind = 0
        if len(birds)>0:
            if len(pipes) > 1 and birds[0].x > pipes[0].x + pipes[0].PIPE_TOP.get_width():
                pipe_ind = 1
                
        else:
            run = False
            break
                
        for x, bird in enumerate(birds):
            bird.move()
            ge[x].fitness += 0.1
            
            output = nets[x].activate((bird.y, abs(bird.y - pipes[pipe_ind].height), abs(bird.y - pipes[pipe_ind].bottom)))
            
            if output[0] > 0.5:
                bird.jump()
                
        #bird.move()
        add_pipe = False
        rem = []
        for pipe in pipes:
            for x, bird in enumerate(birds):
                if pipe.collide(bird):
                    ge[x].fitness -= 1
                    birds.pop(x)
                    nets.pop(x)
                    ge.pop(x)
                
                if not pipe.passed and pipe.x < bird.x:
                    pipe.passed = True
                    add_pipe = True
                    
            if pipe.x + pipe.PIPE_TOP.get_width()<0:
                rem.append(pipe)
                
            pipe.move()
            
        if add_pipe:
            score += 1
            for g in ge:
                g.fitness += 5
            pipes.append(Pipe(600))
            
        for r in rem:
            pipes.remove(r)
            
        for x, bird in enumerate(birds):
            if bird.y + bird.img.get_height() >= 730 or bird.y < 0:
                birds.pop(x)
                nets.pop(x)
                ge.pop(x)
            
            
        base.move()
        draw_screen(screen, birds, pipes, base, score, GEN)
        

        if score > 100:
            break
    
    


In [9]:
#Genetic Algorithm- Python Module- Neat, Neuroevolution of Augmenting Topologies


def run(config_path):
    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_path)
    
    p = neat.Population(config)
    
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    screen = p.run(main,50)
    
    print('\nBest genome:\n{!s}'.format(screen))
    
    pickle.dump(screen,open("best.pickle", "wb"))
    weights = pickle.load(open('best.pickle' , 'rb' ))
        
       
    print(weights)
if __name__ == "__main__":
    local_dir = os.path.dirname('__file__')
    config_path = os.path.join(local_dir, "config-feedforward.txt")
    run(config_path)
    
#Saving the best bird to use the bird
#Pickle, save the object winner as it will return the best genome
#When met the fitness threshhold, pickl it and save it as a file
#Load and use that in your own network associated with the genome to move up and down
#Create a function which runs the bird oly once or you pass one genome
#To get winner, when score touches  50, break, the bird meets fitness threshold and it will quit
#It will return to winner. Pickle it, save it, use it
#Using the neural network draw one bird instead of so many and run


 ****** Running generation 0 ****** 

Population's average fitness: 66.77500 stdev: 276.63716
Best fitness: 1272.60000 - size: (1, 3) - species 1 - id 10

Best individual in generation 0 meets fitness threshold - complexity: (1, 3)

Best genome:
Key: 10
Fitness: 1272.5999999999638
Nodes:
	0 DefaultNodeGene(key=0, bias=-0.060507900478784196, response=1.0, activation=tanh, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-3, 0), weight=-0.3667568900161028, enabled=True)
	DefaultConnectionGene(key=(-2, 0), weight=0.3725823624028095, enabled=True)
	DefaultConnectionGene(key=(-1, 0), weight=-0.04152723846298518, enabled=True)
Key: 10
Fitness: 1272.5999999999638
Nodes:
	0 DefaultNodeGene(key=0, bias=-0.060507900478784196, response=1.0, activation=tanh, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-3, 0), weight=-0.3667568900161028, enabled=True)
	DefaultConnectionGene(key=(-2, 0), weight=0.3725823624028095, enabled=True)
	DefaultConnectionGene(key=(-1, 0), weight=-0.04